In [5]:
import pandas as pd
import json
import ast

In [6]:
import sys, os
sys.path.append(os.path.join(os.path.dirname(sys.path[0]),'src'))
print(sys.path)
from icdlmmeval.codiesp.codiformat import *

['/home/jovyan/work/icdllmeval/notebooks', '/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/opt/conda/lib/python3.10/site-packages', '/home/jovyan/work/icdllmeval/src', '/home/jovyan/work/icdllmeval/src']


In [7]:
df_entities = pd.read_excel('entities-ner.xlsx')
df_entities.head()

,Unnamed: 0,FILE,NR,TEXT,OFFSET,ENTITIES
0,0,S0004-06142005000500011-1,0,"Paciente de 70 años de edad, minero jubilado, ...",0,"[{'entity_group': 'D', 'score': 0.71108776, 'w..."
1,1,S0004-06142005000500011-1,1,En la placa simple de abdomen se valoran cambi...,1056,"[{'entity_group': 'D', 'score': 0.57665336, 'w..."
2,2,S0004-06142005000500011-1,2,La cistoscopia descubre la existencia de peque...,1760,"[{'entity_group': 'D', 'score': 0.96522874, 'w..."
3,3,S0004-06142005000900014-1,0,Varón de diecisiete años sin antecedentes de i...,0,"[{'entity_group': 'D', 'score': 0.98917997, 'w..."
4,4,S0004-06142005000900014-1,1,Realizamos cistoscopia observándose en cara la...,356,"[{'entity_group': 'D', 'score': 0.5218421, 'wo..."


In [8]:
def get_entities(df_entities, file_name):
    df_entities_test = df_entities[df_entities["FILE"] == file_name]
    entities_merged = []
    for idx, row in df_entities_test.iterrows():
        # should become json
        entities = row["ENTITIES"]
        offset = row["OFFSET"]

        entities_py = ast.literal_eval(entities)
        for entity in entities_py:
            entity['start'] = entity['start'] + int(offset)
            entity['end'] = entity['end'] + int(offset)
            entities_merged.append(entity)
    return entities_merged

file_name = "S0004-06142005000500011-1"
entities_merged = get_entities(df_entities=df_entities, file_name=file_name)
print(entities_merged)

[{'entity_group': 'D', 'score': 0.71108776, 'word': ' accidente', 'start': 128, 'end': 137}, {'entity_group': 'D', 'score': 0.96194726, 'word': ' fracturas', 'start': 158, 'end': 167}, {'entity_group': 'D', 'score': 0.9875293, 'word': ' enfermedad', 'start': 207, 'end': 217}, {'entity_group': 'D', 'score': 0.9916488, 'word': ' Diabetes', 'start': 280, 'end': 288}, {'entity_group': 'D', 'score': 0.98975426, 'word': ' hipercolesterolemia', 'start': 307, 'end': 326}, {'entity_group': 'D', 'score': 0.9759991, 'word': ' hiperuricemia', 'start': 329, 'end': 342}, {'entity_group': 'D', 'score': 0.79946584, 'word': ' enolismo', 'start': 344, 'end': 352}, {'entity_group': 'D', 'score': 0.97562397, 'word': ' fumador', 'start': 361, 'end': 368}, {'entity_group': 'D', 'score': 0.99039906, 'word': ' hematuria', 'start': 444, 'end': 453}, {'entity_group': 'D', 'score': 0.86253595, 'word': ' microhematuria', 'start': 498, 'end': 512}, {'entity_group': 'D', 'score': 0.95717716, 'word': ' adenoma', 'st

In [9]:
codiformat = CodiFormat(path_codiesp='/home/jovyan/work/icdllmeval/notebooks/codiesp')
txt = codiformat.get_text('test', file_name)    

In [10]:
print(txt)
for entity in entities_merged:
    entity_s = entity_s=txt[entity['start'] : entity['end']]
    s = f"entity-{entity_s}-entity".format(entity_s)
    print(s)

Paciente de 70 años de edad, minero jubilado, sin alergias medicamentosas conocidas, que presenta como antecedentes personales: accidente laboral antiguo con fracturas vertebrales y costales; intervenido de enfermedad de Dupuytren en mano derecha y by-pass iliofemoral izquierdo; Diabetes Mellitus tipo II, hipercolesterolemia e hiperuricemia; enolismo activo, fumador de 20 cigarrillos / día.
Es derivado desde Atención Primaria por presentar hematuria macroscópica postmiccional en una ocasión y microhematuria persistente posteriormente, con micciones normales.
En la exploración física presenta un buen estado general, con abdomen y genitales normales; tacto rectal compatible con adenoma de próstata grado I/IV.
En la analítica de orina destaca la existencia de 4 hematíes/ campo y 0-5 leucocitos/campo; resto de sedimento normal.
Hemograma normal; en la bioquímica destaca una glucemia de 169 mg/dl y triglicéridos de 456 mg/dl; función hepática y renal normal. PSA de 1.16 ng/ml.
Las citología

In [26]:
def get_type(entity):
    print(entity)
    type = entity["entity_group"]
    if type == "D":
        return DIAGNOSTICO
    else:
        return PROCEDIMIENTO

def get_offset(entity):
    start = entity['start']
    end = entity['end']
    return f"{start} {end}"

header_X = ["FILE","TYPE", "CODE", "SUBSTRING", "OFFSETS"]      

codiesp_x_entities = []
for idx, row in df_entities.iterrows():
    file_name = row["FILE"]
    file_entities = get_entities(df_entities, file_name=file_name)
    for entity in file_entities:
        entity_x = {}
        entity_x["FILE"] = file_name
        entity_x["TYPE"] = get_type(entity)
        entity_x["CODE"] = "-"
        entity_x["SUBSTRING"] = entity['word']
        entity_x["OFFSETS"] = get_offset(entity)
        codiesp_x_entities.append(entity_x)

df_codiesp_x_entities = pd.DataFrame.from_records(codiesp_x_entities, columns=header_X)
df_codiesp_x_entities.head()
        


{'entity_group': 'D', 'score': 0.71108776, 'word': ' accidente', 'start': 128, 'end': 137}
{'entity_group': 'D', 'score': 0.96194726, 'word': ' fracturas', 'start': 158, 'end': 167}
{'entity_group': 'D', 'score': 0.9875293, 'word': ' enfermedad', 'start': 207, 'end': 217}
{'entity_group': 'D', 'score': 0.9916488, 'word': ' Diabetes', 'start': 280, 'end': 288}
{'entity_group': 'D', 'score': 0.98975426, 'word': ' hipercolesterolemia', 'start': 307, 'end': 326}
{'entity_group': 'D', 'score': 0.9759991, 'word': ' hiperuricemia', 'start': 329, 'end': 342}
{'entity_group': 'D', 'score': 0.79946584, 'word': ' enolismo', 'start': 344, 'end': 352}
{'entity_group': 'D', 'score': 0.97562397, 'word': ' fumador', 'start': 361, 'end': 368}
{'entity_group': 'D', 'score': 0.99039906, 'word': ' hematuria', 'start': 444, 'end': 453}
{'entity_group': 'D', 'score': 0.86253595, 'word': ' microhematuria', 'start': 498, 'end': 512}
{'entity_group': 'D', 'score': 0.95717716, 'word': ' adenoma', 'start': 685, 

,FILE,TYPE,CODE,SUBSTRING,OFFSETS
0,S0004-06142005000500011-1,DIAGNOSTICO,-,accidente,128 137
1,S0004-06142005000500011-1,DIAGNOSTICO,-,fracturas,158 167
2,S0004-06142005000500011-1,DIAGNOSTICO,-,enfermedad,207 217
3,S0004-06142005000500011-1,DIAGNOSTICO,-,Diabetes,280 288
4,S0004-06142005000500011-1,DIAGNOSTICO,-,hipercolesterolemia,307 326


In [27]:
df_codiesp_x_entities.to_csv('codiesp_main_ner.tsv', index=False, header=None, sep = '\t')
